In [1]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-06-22 11:53:46--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-22 11:53:46 (38.4 MB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open ('documents.json','rt') as f_in:
    docs_raw= json.load(f_in)

In [5]:
documents= []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course']=course_dict['course']
        documents.append(doc)


In [6]:
documents[0]

{'text': "Anaconda3-2024.02-1-Windows-x86_64.exe\nThe purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index= minsearch.Index(
    text_fields= ["question","text","section"],
    keyword_fields=["course"]
)

In [8]:
#select * where course= 'data-engineering-zoomcamp'

In [9]:
q='the course has already started, can i still enroll?'

In [10]:
index.fit(documents)

In [11]:
boost= {'question':3.0, 'section': 0.5}

results= index.search(
    query=q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [12]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Anaconda3-2024.02-1-Windows-x86_64.exe\nThe purpose of this document is to capture frequently asked technical questions\nThe exact day and

In [14]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

#api_key = os.environ["ZwArrgX9ZCMTcpruMKpEKS2jevNmrn5t"]
model = "mistral-large-latest"

client = MistralClient(api_key="ZwArrgX9ZCMTcpruMKpEKS2jevNmrn5t")

messages = [
    ChatMessage(role="user", content=q)
]

# No streaming
response = client.chat(
    model=model,
    messages=messages,
)

print(response.choices[0].message.content)

That depends on the specific policy of the course or institution. Some courses may allow late enrollment, while others may not. I would recommend reaching out to the course instructor or the institution's administration to ask about their policy on late enrollment. They can provide you with the most accurate information.


In [21]:
prompt_template="""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the QUESTION doesn`t contain the answer, output NONE

QUESTION: {question}
CONTEXT: {context}

""".strip()


In [22]:
context= ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"
    
    ""

In [25]:
prompt= prompt_template.format(question=q, context=context).strip()

In [26]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the QUESTION doesn`t contain the answer, output NONE

QUESTION: the course has already started, can i still enroll?
CONTEXT: section: General course-related questions
question:Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question:Course - Can I follow the course after it finishes?
answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final c

In [27]:
messages = [
    ChatMessage(role="user", content=prompt)
]

# No streaming
response = client.chat(
    model=model,
    messages=messages,
)

print(response.choices[0].message.content)

Yes, you can still enroll in the course even if it has already started. You're eligible to submit homeworks, but be aware of the deadlines for turning in the final projects.


In [69]:
def search (query):
    boost= {'question':3.0, 'section': 0.5}

    results= index.search(
      query=query,
      filter_dict={'course':'data-engineering-zoomcamp'},
      boost_dict=boost,
      num_results=10)
    return results



In [72]:
query="how do i run kafka"
search_results= search(query)

In [73]:
def build_prompt(query, search_results):
    prompt_template="""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the QUESTION doesn`t contain the answer, output 'NONE'

QUESTION: {question}
CONTEXT: {context}

""".strip()
    
    context= ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"
     

    prompt= prompt_template.format(question=query, context=context).strip()
    return prompt


In [74]:
def llm(prompt):
    messages = [ChatMessage(role="user", content=prompt)]

    response = client.chat(
       model=model,
       messages=messages)

    return response.choices[0].message.content
    

In [76]:
query="how do i run kafka"

def rag(query):
    search_results= search(query)
    prompt= build_prompt(query,search_results)
    answer= llm(prompt)
    return answer


In [78]:
rag('how can i submit homework?')

"To submit your homework, you need to follow these steps:\n\n1. Right click and download the pipeline itself to your local machine. This will give you metadata, pycache, and other files.\n2. Download each .py/.sql file that corresponds to each block you created for the pipeline. These files can be found under 'data loaders', 'data transformers', and 'data exporters' in your local src folder.\n3. Move the downloaded files to your GitHub repo folder.\n4. Commit your changes to the GitHub repository.\n\nThe homework URL in the homework link is your repository on GitHub, GitLab, Bitbucket, etc., or any other location where you have your code and where a reasonable person would look at it and think that you have gone through the week and exercises."

In [79]:
documents[0]

{'text': "Anaconda3-2024.02-1-Windows-x86_64.exe\nThe purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [89]:
from elasticsearch import Elasticsearch

In [90]:
es_client= Elasticsearch ('http://localhost:9200')

In [91]:
#es_client.info()

In [92]:
index_settings= {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name='course-questions'
es_client.indices.create(index= index_name, body=index_settings)

Exception ignored in: <function MistralClient.__del__ at 0x70cdaddfcc10>
Traceback (most recent call last):
  File "/usr/local/python/3.10.13/lib/python3.10/site-packages/mistralai/client.py", line 49, in __del__
    self._client.close()
AttributeError: 'MistralClient' object has no attribute '_client'


BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/AiepbGM6T2uNQ0TTfVPGYg] already exists')

In [93]:
from tqdm.auto import tqdm

In [96]:

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 1002/1002 [00:22<00:00, 45.55it/s]


In [119]:
query='How do I execute a command in a running docker container?'

In [121]:
def elastic_search(query):
    search_query= {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }


    response= es_client.search(index=index_name, body= search_query)
    
    results_docs= []
    for hit in response['hits']['hits']:
        results_docs.append(hit['_source'])
    return results_docs

In [129]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'cFgUQJABwvVA5o6BtLiQ',
  '_score': 30.606909,
  '_source': {'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
   'section': 'Workshop 1 - dlthub',
   'question': 'How do I install the necessary dependencies to run the code?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'WVgUQJABwvVA5o6BsriR',
  '_score': 26.801952,
  '_source': {'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
   'section': 'Module 6: streaming with kafka',
   'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-

In [117]:
def rag(query):
    search_results= elastic_search(query)
    prompt= build_prompt(query,search_results)
    answer= llm(prompt)
    return answer

In [118]:
rag("How do I execute a command in a running docker container?")

"To execute a command in a running Docker container, you can use the `docker run` command with the appropriate flags and parameters. Here's an example based on the context provided:\n\n```bash\ndocker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1 pgcli -h pg-database -U root -p 5432 -d ny_taxi\n```\n\nIn this command:\n\n- `docker run` is the command to run a Docker container.\n- `-it` is a flag that tells Docker to open an interactive terminal.\n- `--rm` is a flag that tells Docker to remove the container after it's done running.\n- `--network pg-network` specifies the Docker network to use.\n- `ai2ys/dockerized-pgcli:4.0.1` is the Docker image to use.\n- `pgcli -h pg-database -U root -p 5432 -d ny_taxi` is the command to run inside the Docker container. This specific command is used to connect to a PostgreSQL database using pgcli."